In [1]:
### Libraries for webcam showing in ipynb

In [1]:
from ipywidgets import Video, Image
from IPython.display import display
import IPython
import numpy as np
import cv2
import base64

In [2]:
%pylab inline
from IPython.display import clear_output

Populating the interactive namespace from numpy and matplotlib


## Test RTSP Cam ( Camera over Network)

To proof that system able to run in real condition\
The testing of web cam under headless server (without GUI) will be tested.\
Because CV2 window can't show directly through webpage.

The tutorial that runs cv2 in matplotlib graph are referred:\
[Link to reference](https://github.com/bikz05/ipython-notebooks/blob/master/computer-vision/displaying-video-in-ipython-notebook.ipynb)

In [3]:
# Grab the input device, in this case the webcam

# You can also give path to the video file
vid = cv2.VideoCapture("face-direction-2.mp4")


# Reserve for webcam source
### vid = cv2.VideoCapture(0)

# Put the code in try-except statements
# Catch the keyboard exception and 
# release the camera device and 
# continue with the rest of code.
try:
    # Check if the webcam is opened correctly
    if not vid.isOpened():
        raise IOError("Cannot open webcam")
    
    while(True):
        # Capture frame-by-frame
        ret, frame = vid.read()
        if not ret:
            # Release the Video Device if ret is false
            vid.release()
            # Message to be displayed after releasing the device
            print("Released Video Resource")
            break
        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Turn off the axis
        axis('off')
        # Title of the window
        title("Input Stream")
        # Display the frame
        imshow(frame)
        show()
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device
    print("Released Video Resource")

Released Video Resource


The playing of video may slightly lagging.\
This is because of internet delay and the method refer is changing frame through looping the matplotlib graph showing function.

## Face side using face-net detection

[Source](https://github.com/niconielsen32/ComputerVision/blob/master/headPoseEstimation.py)\
[Video referred](https://www.youtube.com/watch?v=-toNMaS4SeQ)

This method applies face-net function provided in [MediaPipe](https://github.com/google/mediapipe). By extract the landmark from face and calculate the nose pointing position in (x,y).

When Y less than -10 or 10 (Depends on defined thereshold) it will considered the face is facing left or right.\
When X less than -10 or 10 (Depends on defined thereshold) it will considered the face is facing down or up.

In [5]:
# Libraries
import cv2
import mediapipe as mp
import numpy as np
import time

2023-03-08 00:46:46.262670: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 00:46:47.232764: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/azureuser/skt_env/lib/python3.8/site-packages/cv2/../../lib64:
2023-03-08 00:46:47.232865: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/azureuser/skt_env/lib/python3.8/site-packages/cv2/../..

In [6]:
def facePoseInit():
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    return face_mesh, mp_drawing, drawing_spec
    

def facePoseFunction(frame, thereshold, face_mesh_open, mp_drawing, drawing_spec):
    # To improve performance
    frame.flags.writeable = False
    
    # Get the result
    results = face_mesh.process(frame)
    
    # To improve performance
    frame.flags.writeable = True
    
    # Convert the color space from RGB to BGR
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = frame.shape
    face_3d = []
    face_2d = []

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

                    x, y = int(lm.x * img_w), int(lm.y * img_h)

                    # Get the 2D Coordinates
                    face_2d.append([x, y])

                    # Get the 3D Coordinates
                    face_3d.append([x, y, lm.z])       
            
            # Convert it to the NumPy array
            face_2d = np.array(face_2d, dtype=np.float64)

            # Convert it to the NumPy array
            face_3d = np.array(face_3d, dtype=np.float64)

            # The camera matrix
            focal_length = 1 * img_w

            cam_matrix = np.array([ [focal_length, 0, img_h / 2],
                                    [0, focal_length, img_w / 2],
                                    [0, 0, 1]])

            # The distortion parameters
            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            # Solve PnP
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

            # Get rotational matrix
            rmat, jac = cv2.Rodrigues(rot_vec)

            # Get angles
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

            # Get the y rotation degree
            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
          

            # See where the user's head tilting
            if y < -thereshold:
                text = "Looking Left"
            elif y > thereshold:
                text = "Looking Right"
            elif x < -(thereshold*0.2):
                text = "Looking Down"
            elif x > thereshold:
                text = "Looking Up"
            else:
                text = "Forward"

            # Display the nose direction
            nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)

            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (int(nose_2d[0] + y * 10) , int(nose_2d[1] - x * 10))
            
            cv2.line(frame, p1, p2, (255, 0, 0), 3)

            # Add the text on the frame
            cv2.putText(frame, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
            cv2.putText(frame, "x: " + str(np.round(x,2)), (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, "y: " + str(np.round(y,2)), (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, "z: " + str(np.round(z,2)), (500, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


        end = time.time()
        totalTime = end - start

        fps = 1 / totalTime
        #print("FPS: ", fps)

        cv2.putText(frame, f'FPS: {int(fps)}', (20,450), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
        
        
        if face_mesh_open is True:
            mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        #connections=mp_face_mesh.FACEMESH_IRISES,
                        landmark_drawing_spec=drawing_spec
                        #connection_drawing_spec=drawing_spec
            )
        return frame

In [7]:
# video
cap = cv2.VideoCapture("face-position-test-lowfps.mp4")

# webcam
#cap = cv2.VideoCapture(0)


# Put the code in try-except statements
# Catch the keyboard exception and 
# release the camera device and 
# continue with the rest of code.
try:
    # init face side module
    face_mesh, mp_drawing, drawing_spec = facePoseInit()
    
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    
    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        start = time.time()
        
        if not ret:
            # Release the Video Device if ret is false
            cap.release()
            # Message to be displayed after releasing the device
            print("Released Video Resource")
            break
        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # flip the frame also
        frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        
        # frame, thereshold, show facenet?, mp_drawing, drawing_spec
        # DONT TOUCH frame, mp_drawing, drawing_spec
        frame = facePoseFunction(frame, 15 , True, mp_drawing, drawing_spec)
        
        
        ### Codes for showing frame
        # Turn off the axis
        axis('off')
        # Title of the window
        title("Input Stream")
        # Display the frame
        imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        show()
        # Display the frame until new frame is available
        # Overwrite the old graph
        clear_output(wait=True)

# If stop button are clicked
except KeyboardInterrupt:
    # Release the Video Device
    cap.release()
    # Message to be displayed after releasing the device
    print("Released Video Resource")

Released Video Resource


This ables to form two(2) liveness detection test:
1. Turn the face to left and right.
2. Nodding

In [8]:
def faceDirectionFunction(frame, thereshold):
    # To improve performance
    frame.flags.writeable = False
    
    # Get the result
    results = face_mesh.process(frame)
    
    # To improve performance
    frame.flags.writeable = True
    
    # Convert the color space from RGB to BGR
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = frame.shape
    face_3d = []
    face_2d = []

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

                    x, y = int(lm.x * img_w), int(lm.y * img_h)

                    # Get the 2D Coordinates
                    face_2d.append([x, y])

                    # Get the 3D Coordinates
                    face_3d.append([x, y, lm.z])       
            
            # Convert it to the NumPy array
            face_2d = np.array(face_2d, dtype=np.float64)

            # Convert it to the NumPy array
            face_3d = np.array(face_3d, dtype=np.float64)

            # The camera matrix
            focal_length = 1 * img_w

            cam_matrix = np.array([ [focal_length, 0, img_h / 2],
                                    [0, focal_length, img_w / 2],
                                    [0, 0, 1]])

            # The distortion parameters
            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            # Solve PnP
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

            # Get rotational matrix
            rmat, jac = cv2.Rodrigues(rot_vec)

            # Get angles
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

            # Get the y rotation degree
            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
          

            # See where the user's head tilting
            if y < -thereshold:
                direction = "left"
            elif y > thereshold:
                direction = "right"
            elif x < -(thereshold*0.2):
                direction = "down"
            elif x > thereshold:
                direction = "up"
            else:
                direction = "front"
        return direction

Demostration

In [9]:
# video
cap = cv2.VideoCapture("face-direction-3.mp4")
# webcam
#cap = cv2.VideoCapture(0)


# Put the code in try-except statements
# Catch the keyboard exception and 
# release the camera device and 
# continue with the rest of code.
look_left = False
look_right = False
look_front = False
look_down = False

shake = ''
nod = ''

try:
    # init face side module
    face_mesh, mp_drawing, drawing_spec = facePoseInit()
    
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    
    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        start = time.time()
        
        if not ret:
            # Release the Video Device if ret is false
            cap.release()
            # Message to be displayed after releasing the device
            print("Released Video Resource")
            break
        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # flip the frame also
        frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        
        # frame, thereshold
        # DONT TOUCH frame
        direction = faceDirectionFunction(frame, 5)
        
        if direction == "left":
            if look_right == True:
                shake = 'Passed'
                look_left = False
                look_right = False
            look_left = True
        
        if direction == "right":
            if look_left == True:
                shake = 'Passed'
                look_left = False
                look_right = False
            look_right = True
        
        if direction == "front":
            look_front = True
        
        if direction == "down":
            if look_front == True:
                nod = 'Passed'
                look_front = False
                
        
        cv2.putText(frame, "Head shaking test: {}".format(shake), (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, "Nod test: {}".format(nod), (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, direction, (20, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        ### Codes for showing frame
        # Turn off the axis
        axis('off')
        # Title of the window
        title("Input Stream")
        # Display the frame
        imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        show()
        # Display the frame until new frame is available
        # Overwrite the old graph
        clear_output(wait=True)

# If stop button are clicked
except KeyboardInterrupt:
    # Release the Video Device
    cap.release()
    # Message to be displayed after releasing the device
    print("Released Video Resource")

Released Video Resource


## Eye blinking using Landmark

[Source Code](https://github.com/juan-csv/eye_blink_detection/blob/master/eye_blink_detection.py)\
[Article]()

In [10]:
# Libraries
from imutils.video import FileVideoStream
import cv2
import time
import f_detector
import imutils
import numpy as np

In [11]:
# detector init
detector = f_detector.eye_blink_detector()

# counters
COUNTER = 0
TOTAL = 0

# ----------------------------- video -----------------------------
vid = FileVideoStream("./Zhi_Jie/blink-test-1.mp4").start()

try:
    '''
    # Check if the webcam is opened correctly
    if not vid.isOpened():
        raise IOError("Cannot open webcam")
    '''
    
    while(True):
        star_time = time.time()
        im = vid.read()
        im = cv2.flip(im, 1)
        im = imutils.resize(im, width=720)
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        # detectar_rostro    
        rectangles = detector.detector_faces(gray, 0)
        boxes_face = f_detector.convert_rectangles2array(rectangles,im)
        if len(boxes_face)!=0:
            # box on face
            areas = f_detector.get_areas(boxes_face)
            index = np.argmax(areas)
            rectangles = rectangles[index]
            boxes_face = np.expand_dims(boxes_face[index],axis=0)
            # blinks_detector
            COUNTER,TOTAL = detector.eye_blink(gray,rectangles,COUNTER,TOTAL)
            # agregar bounding box
            frame = f_detector.bounding_box(im,boxes_face,['blinks: {}'.format(TOTAL)])
        else:
            frame = im 
        # visualizacion 
        end_time = time.time() - star_time    
        FPS = 1/end_time
        cv2.putText(frame ,f"FPS: {round(FPS,3)}",(10,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
        # Turn off the axis
        axis('off')
        # Title of the window
        title("Input Stream")
        # Display the frame
        imshow(frame)
        show()
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    # Message to be displayed after releasing the device
    print("Released Video Resource")

Released Video Resource


## Face expression detect (Smile)

[Source](https://github.com/Furkan-Gulsen/face-classification)

In [12]:
from IPython.display import HTML

HTML('''<script>
var code_show_err = false;
var code_toggle_err = function() {
    var stderrNodes = document.querySelectorAll('[data-mime-type="application/vnd.jupyter.stderr"]')
    var stderr = Array.from(stderrNodes)
    if (code_show_err){
        stderr.forEach(ele => ele.style.display = 'block');
    } else {
        stderr.forEach(ele => ele.style.display = 'none');
    }
    code_show_err = !code_show_err
}
document.addEventListener('DOMContentLoaded', code_toggle_err);
</script>
To toggle on/off output_stderr, click <a onclick="javascript:code_toggle_err()">here</a>.''')

In [12]:
# Importing required packages
from keras.models import load_model
import numpy as np
import dlib
import cv2

In [13]:
emotion_offsets = (20, 40)
emotions = {
    0: {
        "emotion": "Angry",
        "color": (193, 69, 42)
    },
    1: {
        "emotion": "Disgust",
        "color": (164, 175, 49)
    },
    2: {
        "emotion": "Fear",
        "color": (40, 52, 155)
    },
    3: {
        "emotion": "Happy",
        "color": (23, 164, 28)
    },
    4: {
        "emotion": "Sad",
        "color": (164, 93, 23)
    },
    5: {
        "emotion": "Suprise",
        "color": (218, 229, 97)
    },
    6: {
        "emotion": "Neutral",
        "color": (108, 72, 200)
    }
}


def shapePoints(shape):
    coords = np.zeros((68, 2), dtype="int")
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords


def rectPoints(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)


faceLandmarks = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(faceLandmarks)

emotionModelPath = 'emotionModel.hdf5'  # fer2013_mini_XCEPTION.110-0.65
emotionClassifier = load_model(emotionModelPath, compile=False)
emotionTargetSize = emotionClassifier.input_shape[1:3]

2023-03-08 00:47:19.951023: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 00:47:20.637280: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 50 MB memory:  -> device: 0, name: Tesla M60, pci bus id: 0001:00:00.0, compute capability: 5.2


In [14]:
# video
cap = cv2.VideoCapture("vid.mp4")

# webcam
#cap = cv2.VideoCapture(0)


# Put the code in try-except statements
# Catch the keyboard exception and 
# release the camera device and 
# continue with the rest of code.
try:
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    
    while(True):
        ret, frame = cap.read()
        frame = cv2.resize(frame, (768, 432))

        if not ret:
            break

        grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(grayFrame, 0)
        for rect in rects:
            shape = predictor(grayFrame, rect)
            points = shapePoints(shape)
            (x, y, w, h) = rectPoints(rect)
            grayFace = grayFrame[y:y + h, x:x + w]
            try:
                grayFace = cv2.resize(grayFace, (emotionTargetSize))
            except:
                continue
    
            grayFace = grayFace.astype('float32')
            grayFace = grayFace / 255.0
            grayFace = (grayFace - 0.5) * 2.0
            grayFace = np.expand_dims(grayFace, 0)
            grayFace = np.expand_dims(grayFace, -1)
            emotion_prediction = emotionClassifier.predict(grayFace, verbose = 0)
            emotion_probability = np.max(emotion_prediction)
            if (emotion_probability > 0.36):
                emotion_label_arg = np.argmax(emotion_prediction)
                color = emotions[emotion_label_arg]['color']
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.line(frame, (x, y + h), (x + 20, y + h + 20),
                         color,
                         thickness=2)
                cv2.rectangle(frame, (x + 20, y + h + 20), (x + 110, y + h + 40),
                              color, -1)
                cv2.putText(frame, emotions[emotion_label_arg]['emotion'],
                            (x + 25, y + h + 36), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (255, 255, 255), 1, cv2.LINE_AA)
            else:
                color = (255, 255, 255)
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        
        ### Codes for showing frame
        # Turn off the axis
        axis('off')
        # Title of the window
        title("Input Stream")
        # Display the frame
        imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        show()
        # Display the frame until new frame is available
        # Overwrite the old graph
        clear_output(wait=True)

# If stop button are clicked
except KeyboardInterrupt:
    # Release the Video Device
    cap.release()
    # Message to be displayed after releasing the device
    print("Released Video Resource")

2023-03-08 00:47:27.327693: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } attr { key: "_has_manual_control_dependencies" value { b: true } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla M60" frequency: 1177 num_cores: 16 environment { key: "architecture" value: "5.2" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 2097152 shared_memory_size_per_multiprocessor: 98304 memory_size: 52690944 bandwidth: 160320000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-03-08 00:47:27.397285: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:433] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2023-03-08 00:47:27.397359: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops.c

UnimplementedError: Graph execution error:

Detected at node 'model_1/conv2d_1/Conv2D' defined at (most recent call last):
    File "/anaconda/envs/azureml_py38/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/anaconda/envs/azureml_py38/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/azureuser/skt_env/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/azureuser/skt_env/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/azureuser/skt_env/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/azureuser/skt_env/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/anaconda/envs/azureml_py38/lib/python3.8/

## Mouth open detect

[Source](https://github.com/mauckc/mouth-open/blob/master/detect_open_mouth.py)

In [ ]:
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import imutils
import time
import dlib
import cv2

In [18]:
def mouth_aspect_ratio(mouth):
	# compute the euclidean distances between the two sets of
	# vertical mouth landmarks (x, y)-coordinates
	A = dist.euclidean(mouth[2], mouth[10]) # 51, 59
	B = dist.euclidean(mouth[4], mouth[8]) # 53, 57

	# compute the euclidean distance between the horizontal
	# mouth landmark (x, y)-coordinates
	C = dist.euclidean(mouth[0], mouth[6]) # 49, 55

	# compute the mouth aspect ratio
	mar = (A + B) / (2.0 * C)

	# return the mouth aspect ratio
	return mar

# define one constants, for mouth aspect ratio to indicate open mouth
MOUTH_AR_THRESH = 0.79

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# grab the indexes of the facial landmarks for the mouth
(mStart, mEnd) = (49, 68)

In [20]:
# start the video stream thread
vs = FileVideoStream("mouth.mp4").start()

frame_width = 640
frame_height = 360

# webcam
#cap = VideoStream(0).start


# Put the code in try-except statements
# Catch the keyboard exception and 
# release the camera device and 
# continue with the rest of code.
try:
    
    # loop over frames from the video stream
    while True:
        # grab the frame from the threaded video file stream, resize
        # it, and convert it to grayscale
        # channels)
        frame = vs.read()
        frame = imutils.resize(frame, width=640)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # detect faces in the grayscale frame
        rects = detector(gray, 0)
    
        # loop over the face detections
        for rect in rects:
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # extract the mouth coordinates, then use the
            # coordinates to compute the mouth aspect ratio
            mouth = shape[mStart:mEnd]

            mouthMAR = mouth_aspect_ratio(mouth)
            mar = mouthMAR
            # compute the convex hull for the mouth, then
            # visualize the mouth
            mouthHull = cv2.convexHull(mouth)
		
            cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
            cv2.putText(frame, "MAR: {:.2f}".format(mar), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Draw text if mouth is open
            if mar > MOUTH_AR_THRESH:
                cv2.putText(frame, "Mouth is Open!", (30,60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2)
        # show the frame
        #cv2.imshow("Frame", frame)
        
        ### Codes for showing frame
        # Turn off the axis
        axis('off')
        # Title of the window
        title("Input Stream")
        # Display the frame
        imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        show()
        # Display the frame until new frame is available
        # Overwrite the old graph
        clear_output(wait=True)

# If stop button are clicked
except KeyboardInterrupt:
    # Release the Video Device
    cap.release()
    # Message to be displayed after releasing the device
    print("Released Video Resource")
except AttributeError:
    # Message to be displayed after releasing the device
    print("Released Video Resource")

Released Video Resource
